# Dynamically Select Tools
Use wildcard to dynamically select the right tool to use based on the user's request.

In [ ]:
%pip install agentsjson
%pip install openai

### Setup API keys and references to `agents.json` files
- You need to first setup accounts and apps for the APIs you want to use.
- You can use a service lke Postman to obtain OAuth2 tokens for APIs.
- For Google use: https://developers.google.com/oauthplayground/

In [73]:
OPENAI_API_KEY=""
COLLECTION_NAME=""

RESEND_API_KEY="<your-resend-api-key>"
HUBSPOT_ACCESS_TOKEN="<your-hubspot-access-token>"
GOOGLE_SHEETS_ACCESS_TOKEN="<your-google-sheets-access-token>"

RESEND_AGENTS_JSON_URL="https://raw.githubusercontent.com/wild-card-ai/agents-json/refs/heads/master/agents_json/resend/agents.json"
HUBSPOT_AGENTS_JSON_URL="https://raw.githubusercontent.com/wild-card-ai/agents-json/refs/heads/lang/typescript/agents_json/hubspotcontacts/agents.json"
GOOGLE_SHEETS_AGENTS_JSON_URL="https://raw.githubusercontent.com/wild-card-ai/agents-json/refs/heads/lang/typescript/agents_json/googlesheets/agents.json"


### Load the `agents.json` files

In [74]:
from agentsjson.core.models import Flow
from agentsjson.core.loader import load_agents_json
import json

# load the agents.json file
resend_bundle = load_agents_json(RESEND_AGENTS_JSON_URL)
hubspot_bundle = load_agents_json(HUBSPOT_AGENTS_JSON_URL)
google_sheets_bundle = load_agents_json(GOOGLE_SHEETS_AGENTS_JSON_URL)

# get the flows
resend_flows = resend_bundle.agentsJson.flows
hubspot_flows = hubspot_bundle.agentsJson.flows
google_sheets_flows = google_sheets_bundle.agentsJson.flows

### Setup Authentication

In [75]:
from agentsjson.core.models.auth import OAuth2AuthConfig, ApiKeyAuthConfig, AuthType

# OAuth2 config for HubSpot
hubspot_auth = OAuth2AuthConfig(
    type=AuthType.OAUTH2,
    token=HUBSPOT_ACCESS_TOKEN,
    token_type="Bearer",  # Adding token_type explicitly
    refresh_token=None,
    expires_at=None,
    scopes=None
)

# OAuth2 config for Google Sheets
google_sheets_auth = OAuth2AuthConfig(
    type=AuthType.OAUTH2,
    token=GOOGLE_SHEETS_ACCESS_TOKEN,
    token_type="Bearer",  # Adding token_type explicitly
    refresh_token=None,
    expires_at=None,
    scopes=None
)

# API Key config for Resend
resend_auth = ApiKeyAuthConfig(
    type=AuthType.API_KEY,
    key_value=RESEND_API_KEY,
    key_name=None,
    key_prefix=None
)

### Setup your agent Toolkit

In [76]:
toolkit = {
    "hubspot": {
        "flows": hubspot_flows,
        "bundle": hubspot_bundle,
        "auth": hubspot_auth
    },
    "google sheets": {
        "flows": google_sheets_flows,
        "bundle": google_sheets_bundle,
        "auth": google_sheets_auth
    },
    "resend": {
        "flows": resend_flows,
        "bundle": resend_bundle,
        "auth": resend_auth
    }
}

### Define agent creation function

This is a generic implementation of an agent that can process any natural language request.

In [77]:

from openai import OpenAI
from agentsjson.core import ToolFormat, flows_tools, execute_flows

def create_agent(bundle, flows, system_prompt, auth):
    openai_client = OpenAI(api_key=OPENAI_API_KEY)
    
    def agent(user_request):
        response = openai_client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": user_request}],
            tools=flows_tools(flows, format=ToolFormat.OPENAI),
            tool_choice="auto",
        )
        
        # Run the flows        
        response = execute_flows(response, format=ToolFormat.OPENAI, bundle=bundle, flows=flows, auth=auth)
        return response
    return agent

### Create Wildcard Agent
Create an agent that will access the Wildcard API to find the right tool to use.

In [104]:
wildcard_system_prompt = """You are a wildcard agent that has access to a very special set of tools that can do many things.
Whenever you are given a request analyze whether you need a tool call to accomplish the task or not.
Make a separate request for each unique API you need to call.
If you need a tool call call the wildcard_tool with information about the task to search for the right tool to use.

You must give your arguments for the tool call as Structued Outputs JSON"""


wildcard_tool = {
    "type": "function",
    "function": {
        "name": "wildcard_tool",
        "description": "This function is responsible for calling the wildcard tool search API to find the right tool to use.",
        "parameters": {
            "type": "object",
            "properties": {
                "task": {
                    "type": "string",
                    "description": "A brief description of the task to accomplish described precisely in natural language. Include API name if available. For example: 'Create a new contact in HubSpot'"
                }
            },
            "required": ["task"],
        }
    }
}

### Submit your Query

In [113]:
query = "Get google sheet data for the sheet called My Customer Info"

### Find the Right Tool to Use with Wildcard Agent

In [ ]:
from openai import OpenAI
from agentsjson.core import execute

openai_client = OpenAI(api_key=OPENAI_API_KEY)

messages = [{"role": "system", "content": wildcard_system_prompt}, {"role": "user", "content": query}]

response = openai_client.chat.completions.create(
    model="gpt-4o",  # Fixed model name
    messages=messages,
    tools=[wildcard_tool],
    tool_choice="auto",
    parallel_tool_calls=False
)

print("\nRESPONSE FROM WILDCARD AGENT:")
print(json.dumps(response.model_dump(), indent=2))


### Call the Wildcard Tool Selection API

In [ ]:
import requests

WILDCARD_API_KEY = "<your-wildcard-api-key>"

tool_call = response.choices[0].message.tool_calls[0]

if tool_call.function.name != "wildcard_tool":
    raise ValueError("Tool call is not a wildcard tool call")

# Parse the arguments from the tool call
args = json.loads(tool_call.function.arguments)
task = args["task"]

search_url = "https://queryfd.onrender.com/search"
search_params = {
    "query": task,
    "collection_name": COLLECTION_NAME
}

search_response = requests.get(search_url, params=search_params, headers={"x-api-key": WILDCARD_API_KEY})
search_response.raise_for_status()
search_results = search_response.json()

print(json.dumps(search_results, indent=2))


In [ ]:
flow_id = search_results["points"][0]["payload"]["flow"]["id"]
api_name = search_results["points"][0]["payload"]["info"]["title"]

# Clean and match API name to toolkit keys
api_name = next((key for key in toolkit.keys() if key in api_name.lower()), None)
flow = next((f for f in toolkit[api_name]["flows"] if f.id == flow_id), None)

print("Flow ID:", flow_id)
print("API Name:", api_name)

### Execute the Selected Flow

In [ ]:
# Create an agent to handle the original task using the selected flow
agent = create_agent(
    bundle=toolkit[api_name]["bundle"],
    flows=[flow], # Use just the selected flow
    system_prompt=f"You are an AI assistant that helps users with {api_name} tasks.",
    auth=toolkit[api_name]["auth"]
)

# Execute the original task using the agent
agent_response = agent(query)
print(f"\nAgent response for {api_name} task:")
print(json.dumps(agent_response, indent=2))

### Save the response to the message history and continue the conversation
- You can save the response to the message history and continue the conversation history to make a subsequent request to the agent.
- We suggest include a reflection step to see if there are any follow up questions or tasks to complete.